Assume that we can not reuse any units, so only 1 unit for 1 event, until we have used all 28 units.
The event is sorted by date and then by priority.

In [10]:
import json 
from pathlib import Path  
import pandas as pd 
from dataclasses import dataclass 
from datetime import datetime 
from typing import List, Optional  
from itertools import groupby 
from operator import attrgetter

In [11]:

@dataclass
class Unit:
    """Represents a firefighting resource unit"""
    id: int  
    unit_type: str  
    deployment_time: int  
    cost: float  
    used: bool = False  # Tracks if unit is currently in use

@dataclass
class FireEvent:
    """Represents a wildfire incident"""
    timestamp: datetime  
    start_time: datetime 
    latitude: float 
    longitude: float
    severity: str  
    handled: bool = False  # Whether fire has been addressed
    assigned_unit: Optional[Unit] = None  # Unit assigned to handle the fire
    
    @property
    def date(self):
        """Returns the date of the fire event"""
        return self.timestamp.date()

class ResourceManager:
    """Manages firefighting resources and handles fire events"""
    def __init__(self):
        """Initialize resource manager with units and tracking statistics"""
        self.units = []
        # Dictionary defining unit specifications: (count, deployment_time, cost)
        unit_specs = {
            'smoke_jumper': (5, 30, 5000),
            'fire_engine': (10, 60, 2000),
            'helicopter': (3, 45, 8000),
            'tanker_plane': (2, 120, 15000),
            'ground_crew': (8, 90, 3000)
        }
        
        # Create units based on specifications
        unit_id = 0
        for unit_type, (count, time, cost) in unit_specs.items():
            for _ in range(count):
                self.units.append(Unit(unit_id, unit_type, time, cost))
                unit_id += 1

        # Define damage costs by severity
        self.damage_costs = {'low': 50000, 'medium': 100000, 'high': 200000}
        
        # Initialize statistics tracking
        self.stats = {
            'handled': {'low': 0, 'medium': 0, 'high': 0},
            'missed': {'low': 0, 'medium': 0, 'high': 0},
            'operational_cost': 0,
            'damage_cost': 0,
            'assignments': []
        }

    def get_available_unit(self) -> Optional[Unit]:
        """Find and return first available unit, or None if none available"""
        return next((unit for unit in self.units if not unit.used), None)

    def process_fires(self, fires: List[FireEvent]):
        """Process list of fires, assigning units and updating statistics"""
        severity_rank = {'high': 0, 'medium': 1, 'low': 2}  # Priority ordering
        sorted_fires = sorted(fires, key=attrgetter('date', 'timestamp'))
        
        # Process fires day by day
        for date, day_fires in groupby(sorted_fires, key=attrgetter('date')):
            # Sort daily fires by severity (high priority first)
            daily_fires = sorted(day_fires, key=lambda x: severity_rank[x.severity])
            
            # Process each fire
            for fire in daily_fires:
                unit = self.get_available_unit()
                if unit:  # If unit available, assign it
                    unit.used = True
                    fire.handled = True
                    fire.assigned_unit = unit
                    
                    # Update statistics
                    self.stats['handled'][fire.severity] += 1
                    self.stats['operational_cost'] += unit.cost
                    self.stats['assignments'].append(
                        f"{fire.timestamp.date()}: {fire.severity} fire at ({fire.latitude}, {fire.longitude}) -> {unit.unit_type} (ID: {unit.id})"
                    )
                else:  # If no unit available, record as missed
                    self.stats['missed'][fire.severity] += 1
                    self.stats['damage_cost'] += self.damage_costs[fire.severity]

    def print_report(self):
        """Print comprehensive report of fire handling statistics"""
        # ... (printing formatted statistics) ...

    def save_json_report(self, output_path: str):
        """Save report statistics to JSON file"""
        # Create report dictionary
        report = {
            "summary": {
                "total_fires_handled": sum(self.stats['handled'].values()),
                "total_fires_missed": sum(self.stats['missed'].values()),
                "units_used": sum(1 for unit in self.units if unit.used),
                "total_units": len(self.units)
            },
            "by_severity": {
                "handled": self.stats['handled'],
                "missed": self.stats['missed']
            },
            "costs": {
                "operational": self.stats['operational_cost'],
                "damage": self.stats['damage_cost'],
                "total": self.stats['operational_cost'] + self.stats['damage_cost']
            },
            "assignments": self.stats['assignments'],
            "timestamp": datetime.now().isoformat()
        }

        # Ensure output directory exists
        output_dir = Path(output_path).parent
        output_dir.mkdir(parents=True, exist_ok=True)

        # Write report to JSON file
        with open(output_path, 'w') as f:
            json.dump(report, f, indent=2)

In [12]:
def main():
    """Main function to run the wildfire response simulation"""
    # Define input/output paths
    csv_path = r"C:\Users\taduc\Downloads\SAP-CONUHACKS-IX\Data\Preprocessed_1\current_wildfiredata_cleaned.csv"
    output_path = r"C:\Users\taduc\Downloads\SAP-CONUHACKS-IX\Data\Output\model1_supposedTrue.json"
    
    # Read fire data
    df = pd.read_csv(csv_path)
    
    # Create FireEvent objects from data
    fires = [
        FireEvent(
            timestamp=pd.to_datetime(row['timestamp']),
            start_time=pd.to_datetime(row['fire_start_time']),
            latitude=row['latitude'],
            longitude=row['longitude'],
            severity=row['severity']
        ) for _, row in df.iterrows()
    ]
    
    # Process fires and generate reports
    manager = ResourceManager()
    manager.process_fires(fires)
    manager.print_report()
    manager.save_json_report(output_path)

# Entry point of the script
if __name__ == "__main__":
    main()